In [774]:
import pandas as pd
import sqlite3

In [775]:
file_path = "../data/fifa_players.db"
query_string = "SELECT * FROM prem_name_join"

In [776]:
conn = sqlite3.connect(file_path)

In [777]:
df = pd.read_sql_query(query_string, conn)
df

,player_id,fifa_version,fifa_update,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,...,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,goals,assists
0,192985.0,23,1.0,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91.0,91.0,107500000.0,350000.0,...,74.0,88.0,93.0,87.0,64.0,77.0,94.0,85.0,7,18
1,20801.0,23,1.0,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,ST,90.0,90.0,41000000.0,220000.0,...,81.0,92.0,78.0,85.0,34.0,75.0,80.0,93.0,1,0
2,203376.0,23,1.0,V. van Dijk,Virgil van Dijk,CB,90.0,90.0,98000000.0,230000.0,...,81.0,60.0,71.0,72.0,91.0,86.0,53.0,52.0,3,1
3,209331.0,23,1.0,M. Salah,Mohamed Salah Ghaly,RW,90.0,90.0,115500000.0,270000.0,...,90.0,89.0,82.0,90.0,45.0,75.0,80.0,93.0,19,12
4,200104.0,23,1.0,H. Son,손흥민 孙兴慜,"LW, LM",89.0,89.0,101000000.0,240000.0,...,88.0,89.0,82.0,86.0,42.0,69.0,83.0,91.0,10,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6330,NaN,16,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
6331,NaN,17,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,1
6332,NaN,18,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
6333,NaN,19,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3


### Exploring Null Rows in the Dataset

I'm currently exploring rows in the dataset that are **null in all columns except** `short_name`, `long_name`, `goals`, and `assists`. These nulls are likely the result of incomplete joins during the merge with goal/assist statistics.

Based on how the dataset was built, I’ve assumed that many of these players **joined the Premier League mid-season**. The FIFA data I used comes only from the **first update of each season**, so players who transferred in later might not be present in the FIFA table, but still show up in the goal/assist data scraped from Transfermarkt. 

To investigate these discrepancies:
- I’m conducting **exploratory analysis** of the null rows within the DataFrame to identify patterns and repeated player names (`long_name`) across null entries.
- For players with multiple null rows, I then use **BeeKeeper Studio** to look up their `player_id` and determine if it's possible to accurately attribute goal or assist data.

#### SQL Query Example

To search for a player's data in BeeKeeper Studio, I use queries like:

```sql
SELECT * FROM prem_nlongoin
WHERE short_name LIKE '%capoue%'
```

Where `Etienne Capoue` is just one example of a player being investigated.

---

#### Summary of Findings

- **40 players** have **2 or more null rows**
- **17 players** have **3 or more null rows**

I’m prioritizing players with **2 or more null rows**, as this strongly indicates a join failure due to name mismatch or missing FIFA registration.

Given there are **387 total null rows**, it's likely that some players:
- Only played in the Premier League for **a single season**
- Were not matched correctly due to **name inconsistencies**
- Consequently, had their goals and assists **not attributed** properly

This introduces a potential **bias** in the dataset, especially against short-term players with fewer contributions — who are more likely to be dropped during the join and not matched in scraping.


In [778]:
df_null = df[df['fifa_update'].isnull()]
df_null

,player_id,fifa_version,fifa_update,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,...,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,goals,assists
5948,NaN,22,NaN,A. Lennon,Aaron Lennon,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
5949,NaN,18,NaN,A. Sabiri,Abdelhamid Sabiri,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
5950,NaN,21,NaN,A. Doucouré,Abdoulaye Doucouré,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3
5951,NaN,15,NaN,A. Hernández,Abel Hernández,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1
5952,NaN,17,NaN,A. Traoré,Adama Traoré,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6330,NaN,16,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
6331,NaN,17,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,1
6332,NaN,18,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
6333,NaN,19,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3


In [779]:
null_counts = df_null.groupby('long_name').size()

In [780]:
multi_null_names = null_counts[null_counts >= 2].index
multi_null_names

Index(['Adama Traoré', 'Ademola Lookman', 'Bojan Krkic', 'Branislav Ivanovic',
       'Carlos Vinícius', 'Chicharito', 'Chung-yong Lee', 'Dame N'Doye',
       'Dieumerci Mbokani', 'Gabriel Paulista', 'Gylfi Sigurdsson',
       'Hee-chan Hwang', 'Jay Rodríguez', 'Jeff Hendrick', 'Jonny Otto',
       'Jordan Ayew', 'Jose Cholevas', 'Jóhann Berg Gudmundsson',
       'Karlan Grant', 'Lazar Markovic', 'Luka Milivojevic',
       'Mahmoud Trezeguet', 'Marko Arnautovic', 'Martin Skrtel',
       'Martin Ødegaard', 'Massadio Haidara', 'Mateo Kovacic', 'Modou Barrow',
       'Nemanja Matic', 'Nikica Jelavic', 'Pierre-Emerick Aubameyang',
       'Sead Kolasinac', 'Serge Aurier', 'Sung-yueng Ki', 'Tanguy Ndombélé',
       'Thiago Alcántara', 'Tomas Soucek', 'Wout Weghorst', 'Zanka',
       'Étienne Capoue'],
      dtype='object', name='long_name')

In [781]:
suspicious_players = df[(df['long_name'].isin(multi_null_names)) & (df['fifa_update'].isnull())]
suspicious_players

,player_id,fifa_version,fifa_update,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,...,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,goals,assists
5952,NaN,17,NaN,A. Traoré,Adama Traoré,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
5953,NaN,19,NaN,A. Traoré,Adama Traoré,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
5954,NaN,17,NaN,A. Lookman,Ademola Lookman,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
5955,NaN,21,NaN,A. Lookman,Ademola Lookman,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4
5956,NaN,22,NaN,A. Lookman,Ademola Lookman,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6330,NaN,16,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
6331,NaN,17,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,1
6332,NaN,18,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
6333,NaN,19,NaN,É. Capoue,Étienne Capoue,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3


In [782]:
amount_of_unique_null_players = len(multi_null_names)
amount_of_unique_null_players

40

In [783]:
df[df['player_id']==178213.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
2070,178213.0,Etienne Capoue,20,0,0
2887,178213.0,Etienne Capoue,19,0,0
3564,178213.0,Etienne Capoue,18,0,0
4147,178213.0,Etienne Capoue,17,0,0
4805,178213.0,Etienne Capoue,16,0,0
5413,178213.0,Etienne Capoue,15,0,0


In [784]:
suspicious_players[suspicious_players['long_name'] == "Étienne Capoue"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6330,NaN,Étienne Capoue,16,0,1
6331,NaN,Étienne Capoue,17,7,1
6332,NaN,Étienne Capoue,18,1,0
6333,NaN,Étienne Capoue,19,1,3
6334,NaN,Étienne Capoue,20,0,3


In [785]:
df.loc[4805, ['goals', 'assists']] = [0,1]
df.loc[4147, ['goals', 'assists']] = [7,1]
df.loc[3564, ['goals', 'assists']] = [1,0]
df.loc[2887, ['goals', 'assists']] = [1,3]
df.loc[2070, ['goals', 'assists']] = [0,3]

In [786]:
df[df['player_id']==183491.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
2895,183491.0,Mathias Jattah-Njie Jørgensen,19,0,0
3606,183491.0,Mathias Jattah-Njie Jørgensen,18,0,0


In [787]:
suspicious_players[suspicious_players['long_name'] == "Zanka"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6326,NaN,Zanka,18,0,2
6327,NaN,Zanka,19,3,1


In [788]:
df.loc[2895, ['goals', 'assists']] = [3,1]
df.loc[3606, ['goals', 'assists']] = [0,2]

In [789]:
df[df['player_id']==201153.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
2687,201153.0,Álvaro Borja Morata Martín,19,0,0
3337,201153.0,Álvaro Borja Morata Martín,18,0,0


In [790]:
df.loc[2687, ['goals', 'assists']] = [5,0]
df.loc[3337, ['goals', 'assists']] = [11,6]

Going  down  the  list,  Wout  Weghorst  is  not  in the  database

In [791]:
df[df['player_id']==236792.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
94,236792.0,Tomáš Souček,23,0,0
734,236792.0,Tomáš Souček,22,0,0
1400,236792.0,Tomáš Souček,21,0,0


In [792]:
suspicious_players[suspicious_players['long_name'] == "Tomas Soucek"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6299,NaN,Tomas Soucek,20,3,0
6300,NaN,Tomas Soucek,21,10,1
6301,NaN,Tomas Soucek,22,5,1
6302,NaN,Tomas Soucek,23,2,3


In [793]:
df.loc[1400, ['goals', 'assists']] = [10,1]
df.loc[734, ['goals', 'assists']] = [5,1]
df.loc[94, ['goals', 'assists']] = [2,3]

Soucek  is  an  example  of  both  being  a  late  transfer  and  also  being  a  name  error

In [794]:
df[df['player_id']==189509.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
22,189509.0,Thiago Alcântara do Nascimento,23,0,0
664,189509.0,Thiago Alcântara do Nascimento,22,0,0


In [795]:
suspicious_players[suspicious_players['long_name'] == "Thiago Alcántara"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6289,NaN,Thiago Alcántara,21,1,0
6290,NaN,Thiago Alcántara,22,1,4


In [796]:
df.loc[664, ['goals', 'assists']] = [1,4]

In [797]:
df[df['player_id']==235569.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
165,235569.0,Tanguy Ndombèlé Alvaro,23,0,0
733,235569.0,Tanguy Ndombèlé Alvaro,22,0,0
1428,235569.0,Tanguy Ndombèlé Alvaro,21,0,0
2064,235569.0,Tanguy Ndombèlé Alvaro,20,0,0


In [798]:
suspicious_players[suspicious_players['long_name'] == "Tanguy Ndombélé"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6284,NaN,Tanguy Ndombélé,20,2,2
6285,NaN,Tanguy Ndombélé,21,3,2
6286,NaN,Tanguy Ndombélé,22,1,1


In [799]:
df.loc[733, ['goals', 'assists']] = [1,1]
df.loc[1428, ['goals', 'assists']] = [3,2]
df.loc[2064, ['goals', 'assists']] = [2,2]

In [800]:
df[df['player_id']==180283.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
2220,180283.0,기성용 寄诚庸,20,0,0
2889,180283.0,기성용 寄诚庸,19,0,0
3483,180283.0,기성용 寄诚庸,18,0,0
4148,180283.0,기성용 寄诚庸,17,0,0
4806,180283.0,기성용 寄诚庸,16,0,0
5548,180283.0,기성용 寄诚庸,15,0,0


In [801]:
suspicious_players[suspicious_players['long_name'] == "Sung-yueng Ki"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6278,NaN,Sung-yueng Ki,15,8,1
6279,NaN,Sung-yueng Ki,16,2,1
6280,NaN,Sung-yueng Ki,17,0,1
6281,NaN,Sung-yueng Ki,18,2,2
6282,NaN,Sung-yueng Ki,19,0,1


In [802]:
df.loc[2889, ['goals', 'assists']] = [0,1]
df.loc[3483, ['goals', 'assists']] = [2,2]
df.loc[4148, ['goals', 'assists']] = [0,1]
df.loc[4806, ['goals', 'assists']] = [2,1]
df.loc[5548, ['goals', 'assists']] = [8,1]

In [803]:
df[df['player_id']==186452.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
3568,186452.0,Siem Stefan de Jong,18,0,0
4812,186452.0,Siem Stefan de Jong,16,0,0
5444,186452.0,Siem Stefan de Jong,15,0,0


In [804]:
df.loc[4812, ['goals', 'assists']] = [0,1]
df.loc[5444, ['goals', 'assists']] = [4,1]

In [805]:
df[df['player_id']==197853.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
759,197853.0,Serge Alain Stéphane Aurier,22,0,0
1410,197853.0,Serge Alain Stéphane Aurier,21,2,3
2108,197853.0,Serge Alain Stéphane Aurier,20,1,5
2731,197853.0,Serge Alain Stéphane Aurier,19,0,2


In [806]:
suspicious_players[suspicious_players['long_name'] == "Serge Aurier"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6267,NaN,Serge Aurier,18,2,2
6268,NaN,Serge Aurier,23,1,0


These  seasons  Aurier  must  have  joined  late

In [807]:
df[df['player_id']==207993.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
971,207993.0,Sead Kolašinac,22,0,0
1478,207993.0,Sead Kolašinac,21,0,0
2118,207993.0,Sead Kolašinac,20,0,0
2769,207993.0,Sead Kolašinac,19,0,0
3439,207993.0,Sead Kolašinac,18,0,0


In [808]:
suspicious_players[suspicious_players['long_name'] == "Sead Kolasinac"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6264,NaN,Sead Kolasinac,18,2,4
6265,NaN,Sead Kolasinac,19,0,6
6266,NaN,Sead Kolasinac,20,0,2


In [809]:
df.loc[3439, ['goals', 'assists']] = [2,4]
df.loc[2769, ['goals', 'assists']] = [0,6]
df.loc[2118, ['goals', 'assists']] = [0,2]

In [810]:
df[df['player_id']==175932.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
5019,175932.0,Ritchie Ria Alfons De Laet,16,0,0
5704,175932.0,Ritchie Ria Alfons De Laet,15,0,0


In [811]:
df.loc[5019, ['goals', 'assists']] = [1,0]
df.loc[5704, ['goals', 'assists']] = [0,1]

In [812]:
df[df['player_id']==242641.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
289,242641.0,Rayan Aït Nouri,23,0,0
1067,242641.0,Rayan Aït Nouri,22,0,0


In [813]:
df.loc[289, ['goals', 'assists']] = [1,0]
df.loc[1067, ['goals', 'assists']] = [1,4]

In [814]:
df[df['player_id']==186190.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
1501,186190.0,Patrick John Miguel van Aanholt,21,0,0
2224,186190.0,Patrick John Miguel van Aanholt,20,0,0
2944,186190.0,Patrick John Miguel van Aanholt,19,0,0
3608,186190.0,Patrick John Miguel van Aanholt,18,0,0
4284,186190.0,Patrick John Miguel van Aanholt,17,0,0
4949,186190.0,Patrick John Miguel van Aanholt,16,0,0
5651,186190.0,Patrick John Miguel van Aanholt,15,0,0


In [815]:
df.loc[1501, ['goals', 'assists']] = [0,1]
df.loc[2224, ['goals', 'assists']] = [3,2]
df.loc[2944, ['goals', 'assists']] = [3,2]
df.loc[3608, ['goals', 'assists']] = [5,1]
df.loc[4284, ['goals', 'assists']] = [5,1]
df.loc[4949, ['goals', 'assists']] = [5,4]
df.loc[5651, ['goals', 'assists']] = [0,5]

In [816]:
df[df['player_id']==191202.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
781,191202.0,Nemanja Matić,22,0,0
1408,191202.0,Nemanja Matić,21,0,0
2052,191202.0,Nemanja Matić,20,0,0
2652,191202.0,Nemanja Matić,19,0,0
3350,191202.0,Nemanja Matić,18,0,0
4003,191202.0,Nemanja Matić,17,0,0
4647,191202.0,Nemanja Matić,16,0,0
5328,191202.0,Nemanja Matić,15,0,0


In [817]:
suspicious_players[suspicious_players['long_name'] == "Nemanja Matic"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6219,NaN,Nemanja Matic,15,1,3
6220,NaN,Nemanja Matic,16,2,2
6221,NaN,Nemanja Matic,17,1,7
6222,NaN,Nemanja Matic,18,1,1
6223,NaN,Nemanja Matic,19,1,0
6224,NaN,Nemanja Matic,20,0,2
6225,NaN,Nemanja Matic,21,0,1
6226,NaN,Nemanja Matic,22,0,4


In [818]:
df.loc[5328, ['goals', 'assists']] = [1,3]
df.loc[4647, ['goals', 'assists']] = [2,2]
df.loc[4003, ['goals', 'assists']] = [1,7]
df.loc[3350, ['goals', 'assists']] = [1,1]
df.loc[2652, ['goals', 'assists']] = [1,0]
df.loc[2052, ['goals', 'assists']] = [0,2]
df.loc[1408, ['goals', 'assists']] = [0,1]
df.loc[781, ['goals', 'assists']] = [0,4]

In [819]:
df[df['player_id']==207410.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
40,207410.0,Mateo Kovačić,23,0,0
709,207410.0,Mateo Kovačić,22,0,0
1363,207410.0,Mateo Kovačić,21,0,0
2042,207410.0,Mateo Kovačić,20,0,0


In [820]:
suspicious_players[suspicious_players['long_name'] == "Mateo Kovacic"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6193,NaN,Mateo Kovacic,19,0,2
6194,NaN,Mateo Kovacic,20,1,3
6195,NaN,Mateo Kovacic,21,0,1
6196,NaN,Mateo Kovacic,22,2,5
6197,NaN,Mateo Kovacic,23,1,2


In [821]:
df.loc[40, ['goals', 'assists']] = [1,2]
df.loc[709, ['goals', 'assists']] = [2,5]
df.loc[1363, ['goals', 'assists']] = [0,1]
df.loc[2042, ['goals', 'assists']] = [1,3]

In [822]:
df[df['player_id']==201955.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
3761,201955.0,Massadio Haïdara,18,0,0
5031,201955.0,Massadio Haïdara,16,0,0
5718,201955.0,Massadio Haïdara,15,0,0


In [823]:
suspicious_players[suspicious_players['long_name'] == "Massadio Haidara"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6190,NaN,Massadio Haidara,15,0,1
6191,NaN,Massadio Haidara,16,0,1


In [824]:
df.loc[5718, ['goals', 'assists']] = [0,1]
df.loc[5031, ['goals', 'assists']] = [0,1]

In [825]:
df[df['player_id']==222665.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
43,222665.0,Martin Ødegaard,23,15,7


In [826]:
suspicious_players[suspicious_players['long_name'] == "Martin Ødegaard"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6186,NaN,Martin Ødegaard,21,1,2
6187,NaN,Martin Ødegaard,22,7,4


**Honestly**, not sure why Ødegaard is disregarded from the database for two seasons.  
I specified that players play in the *Prem*, and he is clearly part of **Arsenal**.  
Maybe he had a different player ID?

In [827]:
df[df['player_id']==166706.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
4676,166706.0,Martin Škrtel,16,0,0
5337,166706.0,Martin Škrtel,15,0,0


In [828]:
suspicious_players[suspicious_players['long_name'] == "Martin Skrtel"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6184,NaN,Martin Skrtel,15,1,0
6185,NaN,Martin Skrtel,16,1,0


In [829]:
df.loc[4676, ['goals', 'assists']] = [1,0]
df.loc[5337, ['goals', 'assists']] = [1,0]

In [830]:
df[df['player_id']==184200.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
2700,184200.0,Marko Arnautović,19,0,0
3372,184200.0,Marko Arnautović,18,0,0
4030,184200.0,Marko Arnautović,17,0,0
4771,184200.0,Marko Arnautović,16,0,0
5441,184200.0,Marko Arnautović,15,0,0


In [831]:
suspicious_players[suspicious_players['long_name'] == "Marko Arnautovic"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6178,NaN,Marko Arnautovic,15,1,5
6179,NaN,Marko Arnautovic,16,11,6
6180,NaN,Marko Arnautovic,17,6,5
6181,NaN,Marko Arnautovic,18,11,6
6182,NaN,Marko Arnautovic,19,10,4


In [832]:
df.loc[5441, ['goals', 'assists']] = [1,5]
df.loc[4771, ['goals', 'assists']] = [11,6]
df.loc[4030, ['goals', 'assists']] = [6,5]
df.loc[3372, ['goals', 'assists']] = [11,6]
df.loc[2700, ['goals', 'assists']] = [10,4]

In [833]:
df[df['player_id']==206304.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
373,206304.0,Luka Milivojević,23,0,0
912,206304.0,Luka Milivojević,22,0,0
1442,206304.0,Luka Milivojević,21,0,0
2057,206304.0,Luka Milivojević,20,0,0
2796,206304.0,Luka Milivojević,19,0,0
3545,206304.0,Luka Milivojević,18,0,0


In [834]:
suspicious_players[suspicious_players['long_name'] == "Luka Milivojevic"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6160,NaN,Luka Milivojevic,17,2,0
6161,NaN,Luka Milivojevic,18,10,1
6162,NaN,Luka Milivojevic,19,12,2
6163,NaN,Luka Milivojevic,20,3,1
6164,NaN,Luka Milivojevic,21,1,1


In [835]:
df.loc[3545, ['goals', 'assists']] = [10,1]
df.loc[2796, ['goals', 'assists']] = [12,2]
df.loc[2057, ['goals', 'assists']] = [3,1]
df.loc[1442, ['goals', 'assists']] = [1,1]

In [836]:
df[df['player_id']==200949.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
104,200949.0,Lucas Rodrigues Moura da Silva,23,1,0
105,200949.0,Lucas Rodrigues Moura da Silva,23,2,6
738,200949.0,Lucas Rodrigues Moura da Silva,22,2,6
739,200949.0,Lucas Rodrigues Moura da Silva,22,7,2
1360,200949.0,Lucas Rodrigues Moura da Silva,21,3,4
1361,200949.0,Lucas Rodrigues Moura da Silva,21,2,2
2020,200949.0,Lucas Rodrigues Moura da Silva,20,4,4
2021,200949.0,Lucas Rodrigues Moura da Silva,20,3,2
2709,200949.0,Lucas Rodrigues Moura da Silva,19,0,0


In [837]:
df.loc[2709, ['goals', 'assists']] = [10,1]

In [838]:
df[df['player_id']==212125.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
2968,212125.0,Lazar Marković,19,0,0
3586,212125.0,Lazar Marković,18,0,0
4176,212125.0,Lazar Marković,17,0,0
4828,212125.0,Lazar Marković,16,0,0
5424,212125.0,Lazar Marković,15,0,0


In [839]:
suspicious_players[suspicious_players['long_name'] == "Lazar Markovic"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6150,NaN,Lazar Markovic,15,2,1
6151,NaN,Lazar Markovic,17,2,0


In [840]:
df.loc[5424, ['goals', 'assists']] = [2,1]
df.loc[4176, ['goals', 'assists']] = [2,0]

In [841]:
df[df['player_id']==191076.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
955,191076.0,Johann Berg Guðmunds­son,22,0,0
1510,191076.0,Johann Berg Guðmunds­son,21,0,0
2152,191076.0,Johann Berg Guðmunds­son,20,0,0
2786,191076.0,Johann Berg Guðmunds­son,19,0,0
3683,191076.0,Johann Berg Guðmunds­son,18,0,0
4368,191076.0,Johann Berg Guðmunds­son,17,1,2


In [842]:
suspicious_players[suspicious_players['long_name'] == "Jóhann Berg Gudmundsson"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6131,NaN,Jóhann Berg Gudmundsson,18,2,8
6132,NaN,Jóhann Berg Gudmundsson,19,3,6
6133,NaN,Jóhann Berg Gudmundsson,20,1,1
6134,NaN,Jóhann Berg Gudmundsson,21,2,0
6135,NaN,Jóhann Berg Gudmundsson,22,0,1


In [843]:
df.loc[3683, ['goals', 'assists']] = [2,8]
df.loc[2786, ['goals', 'assists']] = [3,6]
df.loc[2152, ['goals', 'assists']] = [1,1]
df.loc[1510, ['goals', 'assists']] = [2,0]
df.loc[955, ['goals', 'assists']] = [0,1]

In [844]:
df[df['player_id']==182744.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
2141,182744.0,José Holebas,20,0,0
2891,182744.0,José Holebas,19,0,0
3484,182744.0,José Holebas,18,0,0
4197,182744.0,José Holebas,17,0,0
4807,182744.0,José Holebas,16,0,0


In [845]:
suspicious_players[suspicious_players['long_name'] == "Jose Cholevas"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6119,NaN,Jose Cholevas,16,1,0
6120,NaN,Jose Cholevas,17,2,4
6121,NaN,Jose Cholevas,18,0,4
6122,NaN,Jose Cholevas,19,3,6


In [846]:
df.loc[4807, ['goals', 'assists']] = [1,0]
df.loc[4197, ['goals', 'assists']] = [2,4]
df.loc[3484, ['goals', 'assists']] = [0,4]
df.loc[2891, ['goals', 'assists']] = [3,6]

In [847]:
df[df['player_id']==197756.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
306,197756.0,Jordan Pierre Ayew,23,4,3
957,197756.0,Jordan Pierre Ayew,22,3,3
1515,197756.0,Jordan Pierre Ayew,21,1,3
2290,197756.0,Jordan Pierre Ayew,20,9,2
3574,197756.0,Jordan Pierre Ayew,18,7,2
4863,197756.0,Jordan Pierre Ayew,16,7,0


In [848]:
suspicious_players[suspicious_players['long_name'] == "Jordan Ayew"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6117,NaN,Jordan Ayew,17,1,3
6118,NaN,Jordan Ayew,19,1,2


These  seasons  are  not  in  the  database

In [849]:
df[df['player_id']==210455.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
149,210455.0,Jonathan Castro Otto,23,0,0
761,210455.0,Jonathan Castro Otto,22,0,0
1391,210455.0,Jonathan Castro Otto,21,0,0
2121,210455.0,Jonathan Castro Otto,20,0,0
2799,210455.0,Jonathan Castro Otto,19,0,0


In [850]:
suspicious_players[suspicious_players['long_name'] == "Jonny Otto"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6113,NaN,Jonny Otto,19,1,1
6114,NaN,Jonny Otto,20,2,2
6115,NaN,Jonny Otto,22,2,0
6116,NaN,Jonny Otto,23,1,0


In [851]:
df.loc[149, ['goals', 'assists']] = [1,0]
df.loc[761, ['goals', 'assists']] = [2,0]
df.loc[2121, ['goals', 'assists']] = [2,2]
df.loc[2799, ['goals', 'assists']] = [1,1]

In [852]:
df[df['player_id']==200478.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
1008,200478.0,Jeff Patrick Hendrick,22,1,0
2294,200478.0,Jeff Patrick Hendrick,20,2,2
2957,200478.0,Jeff Patrick Hendrick,19,3,0
3624,200478.0,Jeff Patrick Hendrick,18,2,2


In [853]:
suspicious_players[suspicious_players['long_name'] == "Jeff Hendrick"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6103,NaN,Jeff Hendrick,17,2,1
6104,NaN,Jeff Hendrick,21,2,1


Joined  late  that  season

In [854]:
df[df['player_id']==169792.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
994,169792.0,Jay Rodriguez,22,7,2
1546,169792.0,Jay Rodriguez,21,2,2
2217,169792.0,Jay Rodriguez,20,3,2
3562,169792.0,Jay Rodriguez,18,0,0
4192,169792.0,Jay Rodriguez,17,0,0
4803,169792.0,Jay Rodriguez,16,0,0
5508,169792.0,Jay Rodriguez,15,0,0


In [855]:
suspicious_players[suspicious_players['long_name'] == "Jay Rodríguez"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6096,NaN,Jay Rodríguez,17,5,2
6097,NaN,Jay Rodríguez,18,7,1
6098,NaN,Jay Rodríguez,20,8,2
6099,NaN,Jay Rodríguez,21,1,2


In [856]:
df.loc[1546, ['goals', 'assists']] = [1,2]
df.loc[2217, ['goals', 'assists']] = [8,2]
df.loc[3562, ['goals', 'assists']] = [7,1]
df.loc[4192, ['goals', 'assists']] = [5,2]

In [857]:
df[df['player_id']==226380.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
224,226380.0,황희찬 黄喜灿,23,0,0


Hee-chan Hwang

In [858]:
suspicious_players[suspicious_players['long_name'] == "Hee-chan Hwang"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6083,NaN,Hee-chan Hwang,22,5,0
6084,NaN,Hee-chan Hwang,23,3,1


In [859]:
df.loc[224, ['goals', 'assists']] = [3,1]

In [860]:
df[df['player_id']==184484.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
777,184484.0,Gylfi Þór Sigurðsson,22,0,0
1435,184484.0,Gylfi Þór Sigurðsson,21,0,0
2016,184484.0,Gylfi Þór Sigurðsson,20,0,0
2701,184484.0,Gylfi Þór Sigurðsson,19,0,0
3373,184484.0,Gylfi Þór Sigurðsson,18,0,0
4031,184484.0,Gylfi Þór Sigurðsson,17,0,0
4772,184484.0,Gylfi Þór Sigurðsson,16,0,0
5442,184484.0,Gylfi Þór Sigurðsson,15,0,0


In [861]:
suspicious_players[suspicious_players['long_name'] == "Gylfi Sigurdsson"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6073,NaN,Gylfi Sigurdsson,15,7,10
6074,NaN,Gylfi Sigurdsson,16,11,4
6075,NaN,Gylfi Sigurdsson,17,9,13
6076,NaN,Gylfi Sigurdsson,18,4,3
6077,NaN,Gylfi Sigurdsson,19,13,6
6078,NaN,Gylfi Sigurdsson,20,2,3
6079,NaN,Gylfi Sigurdsson,21,6,5


In [862]:
df.loc[1435, ['goals', 'assists']] = [3,1]
df.loc[2016, ['goals', 'assists']] = [3,1]
df.loc[2701, ['goals', 'assists']] = [3,1]
df.loc[3373, ['goals', 'assists']] = [4,3]
df.loc[4031, ['goals', 'assists']] = [13,6]
df.loc[4772, ['goals', 'assists']] = [2,3]
df.loc[5442, ['goals', 'assists']] = [6,5]

In [863]:
df[df['player_id']==201305.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
4129,201305.0,Gabriel Armando de Abreu,17,0,0
4780,201305.0,Gabriel Armando de Abreu,16,0,0


In [864]:
suspicious_players[suspicious_players['long_name'] == "Gabriel Paulista"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6062,NaN,Gabriel Paulista,15,0,1
6063,NaN,Gabriel Paulista,16,1,0


In [865]:
df.loc[4780, ['goals', 'assists']] = [1,0]

In [866]:
df[df['player_id']==232580.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
92,232580.0,Gabriel dos Santos Magalhães,23,0,0
809,232580.0,Gabriel dos Santos Magalhães,22,0,0


In [867]:
df.loc[809, ['goals', 'assists']] = [5,0]
df.loc[92, ['goals', 'assists']] = [3,0]

In [868]:
df[df['player_id']==162131.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
2845,162131.0,Fernando Javier Llorente Torres,19,0,0
3479,162131.0,Fernando Javier Llorente Torres,18,0,0
4107,162131.0,Fernando Javier Llorente Torres,17,0,0


In [869]:
df.loc[2845, ['goals', 'assists']] = [1,4]
df.loc[3479, ['goals', 'assists']] = [1,0]
df.loc[4107, ['goals', 'assists']] = [15,1]

In [870]:
df[df['player_id']==201118.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
308,201118.0,Cédric Ricardo Alves Soares,23,0,0
907,201118.0,Cédric Ricardo Alves Soares,22,0,0
1562,201118.0,Cédric Ricardo Alves Soares,21,0,0
2238,201118.0,Cédric Ricardo Alves Soares,20,0,0
2866,201118.0,Cédric Ricardo Alves Soares,19,0,0
3465,201118.0,Cédric Ricardo Alves Soares,18,0,0
4166,201118.0,Cédric Ricardo Alves Soares,17,0,0
4779,201118.0,Cédric Ricardo Alves Soares,16,0,0


In [871]:
df.loc[2866, ['goals', 'assists']] = [1,2]
df.loc[2238, ['goals', 'assists']] = [1,1]
df.loc[1562, ['goals', 'assists']] = [0,1]
df.loc[907, ['goals', 'assists']] = [1,1]

In [872]:
df[df['player_id']==155355.0][['player_id', 'short_name','long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
3710,155355.0,Lee Chung Yong,이청용 李青龙,18,0,0
4304,155355.0,Lee Chung Yong,이청용 李青龙,17,0,0
4983,155355.0,Lee Chung Yong,이청용 李青龙,16,0,0


In [873]:
suspicious_players[suspicious_players['long_name'] == "Chung-yong Lee"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6013,NaN,Chung-yong Lee,16,1,0
6014,NaN,Chung-yong Lee,17,0,1


In [874]:
df.loc[4983, ['goals', 'assists']] = [1,0]
df.loc[4304, ['goals', 'assists']] = [0,1]

In [875]:
df[df['player_id']==178224.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
2140,178224.0,Javier Hernández Balcázar,20,0,0
2784,178224.0,Javier Hernández Balcázar,19,0,0
3368,178224.0,Javier Hernández Balcázar,18,0,0
4707,178224.0,Javier Hernández Balcázar,16,0,0
5338,178224.0,Javier Hernández Balcázar,15,0,0


In [876]:
suspicious_players[suspicious_players['long_name'] == "Chicharito"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
6007,NaN,Chicharito,18,8,0
6008,NaN,Chicharito,19,7,1
6009,NaN,Chicharito,20,1,0


In [877]:
df.loc[3368, ['goals', 'assists']] = [8,0]
df.loc[2784, ['goals', 'assists']] = [7,1]
df.loc[2140, ['goals', 'assists']] = [1,0]

In [878]:
df[df['player_id']==178372.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
4085,178372.0,Branislav Ivanović,17,0,0
4708,178372.0,Branislav Ivanović,16,0,0
5339,178372.0,Branislav Ivanović,15,0,0


In [879]:
suspicious_players[suspicious_players['long_name'] == "Branislav Ivanovic"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
5988,NaN,Branislav Ivanovic,15,4,6
5989,NaN,Branislav Ivanovic,16,2,2


In [880]:
df.loc[5339, ['goals', 'assists']] = [4,6]
df.loc[4708, ['goals', 'assists']] = [2,2]

In [881]:
df[df['player_id']==176993.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
3451,176993.0,Bojan Krkić Pérez,18,0,0
4025,176993.0,Bojan Krkíc Pérez,17,0,0
4678,176993.0,Bojan Krkíc Pérez,16,0,0
5438,176993.0,Bojan Krkíc Pérez,15,0,0


In [882]:
suspicious_players[suspicious_players['long_name'] == "Bojan Krkic"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
5984,NaN,Bojan Krkic,15,4,1
5985,NaN,Bojan Krkic,16,7,1
5986,NaN,Bojan Krkic,17,3,0


In [883]:
df.loc[5438, ['goals', 'assists']] = [4,1]
df.loc[4678, ['goals', 'assists']] = [7,1]
df.loc[4025, ['goals', 'assists']] = [3,0]

In [884]:
df[df['player_id']==191005.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
1733,191005.0,Ahmed Eissa El Mohamady Abdel Fattah,21,0,0
2442,191005.0,Ahmed Eissa El Mohamady Abdel Fattah,20,0,0
4290,191005.0,Ahmed Eissa El Mohamady Abdel Fattah,17,0,0
5627,191005.0,Ahmed Eissa El Mohamady Abdel Fattah,15,0,0


In [885]:
df.loc[5627, ['goals', 'assists']] = [2,5]
df.loc[4290, ['goals', 'assists']] = [0,2]
df.loc[2442, ['goals', 'assists']] = [1,1]

In [886]:
df[df['player_id']==230899.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
3102,230899.0,Ademola Lookman,19,0,2
3850,230899.0,Ademola Lookman,18,0,0


In [887]:
suspicious_players[suspicious_players['long_name'] == "Ademola Lookman"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
5954,NaN,Ademola Lookman,17,1,0
5955,NaN,Ademola Lookman,21,4,4
5956,NaN,Ademola Lookman,22,6,0


In [888]:
df[df['player_id']==213956.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
151,213956.0,Adama Traoré Diarra,23,2,2
833,213956.0,Adama Traoré Diarra,22,0,0
1447,213956.0,Adama Traoré Diarra,21,2,3
2350,213956.0,Adama Traoré Diarra,20,4,9
5064,213956.0,Adama Traoré Diarra,16,0,1


In [889]:
suspicious_players[suspicious_players['long_name'] == "Adama Traoré"][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
5952,NaN,Adama Traoré,17,0,1
5953,NaN,Adama Traoré,19,1,1


In [890]:
df = df[df['fifa_update'].notna()]

In [891]:
df[df['player_id']==135507.0]

,player_id,fifa_version,fifa_update,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,...,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,goals,assists
699,135507.0,22,1.0,Fernandinho,Fernando Luiz Rosa,"CDM, CB",83.0,83.0,7000000.0,100000.0,...,59.0,72.0,75.0,78.0,84.0,75.0,68.0,66.0,2,1
1346,135507.0,21,1.0,Fernandinho,Fernando Luiz Rosa,"CB, CDM",84.0,84.0,11000000.0,110000.0,...,64.0,72.0,78.0,78.0,85.0,79.0,68.0,66.0,0,2
1986,135507.0,20,1.0,Fernandinho,Fernando Luiz Rosa,CDM,87.0,87.0,19500000.0,200000.0,...,66.0,74.0,79.0,78.0,84.0,79.0,68.0,69.0,0,1
2649,135507.0,19,1.0,Fernandinho,Fernando Luiz Rosa,CDM,86.0,86.0,18000000.0,180000.0,...,70.0,74.0,77.0,79.0,83.0,81.0,68.0,69.0,1,3
3359,135507.0,18,1.0,Fernandinho,Fernando Luiz Rosa,"CDM, CM, RB",82.0,82.0,12500000.0,130000.0,...,73.0,73.0,76.0,78.0,77.0,78.0,68.0,69.0,5,3
4046,135507.0,17,1.0,Fernandinho,Fernando Luiz Rosa,"CM, CDM",81.0,81.0,14000000.0,130000.0,...,77.0,75.0,77.0,78.0,76.0,78.0,68.0,70.0,2,1
4694,135507.0,16,1.0,Fernandinho,Fernando Luiz Rosa,"CM, CDM",80.0,80.0,12500000.0,100000.0,...,77.0,74.0,78.0,80.0,74.0,77.0,77.0,70.0,2,3
4695,135507.0,16,1.0,Fernandinho,Fernando Luiz Rosa,"CM, CDM",80.0,80.0,12500000.0,100000.0,...,77.0,74.0,78.0,80.0,74.0,77.0,77.0,70.0,2,0
5317,135507.0,15,1.0,Fernandinho,Fernando Luiz Rosa,"CM, CDM",82.0,82.0,14500000.0,130000.0,...,78.0,74.0,82.0,81.0,73.0,75.0,78.0,70.0,3,4
5318,135507.0,15,1.0,Fernandinho,Fernando Luiz Rosa,"CM, CDM",82.0,82.0,14500000.0,130000.0,...,78.0,74.0,82.0,81.0,73.0,75.0,78.0,70.0,2,2


In [892]:
df = df.drop([5318, 4695])

### Cleaning Goals/Assists Data

The SQL join I used to connect goals and assists stats to FIFA players was intentionally **overfitted**. This was necessary because some players were not being correctly matched — their names in the FIFA database were in other languages or used full names that didn't match the format on Transfermarkt (where I sourced the stats).

However, this overfitting also introduced a new issue: **false-positive matches**. Some players were incorrectly assigned goals or assists because their names **partially matched** another player’s name. A key example is the player **Fred** (a Brazilian midfielder for Manchester United). Because of fuzzy matching logic, stats meant for Fred may have been incorrectly attributed to players with names like **Frederick**, **Alfred**, or others containing "Fred" within their `long_name`.

This happened due to the `LIKE`-based logic in the join, where the SQL used conditions such as:

```sql
LOWER(fp.long_name) LIKE '%' || LOWER(cp.long_name) || '%'
OR LOWER(cp.long_name) LIKE '%' || LOWER(fp.long_name) || '%'
```

While this helped catch mismatches due to name formatting or language differences, it also meant that short or common names embedded in other names could cause misattribution.

---

Using BeeKeeper Studio, I found:
- **98 rows** where players were assigned goals or assists **more than once**
- These duplicates involved only **24 unique players** that needed correction

I manually looked up each of these players' Premier League goals and assists for each season and wrote them down. This allowed me to remove incorrect duplicates from the DataFrame.

As a result, the data should now be much **cleaner** and less prone to name-matching errors.

**Example:** Fernandinho was the first player I corrected.

 corrected.


In [893]:
df[df['player_id']==200054.0]

,player_id,fifa_version,fifa_update,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,...,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,goals,assists
2864,200054.0,19,1.0,Pedro Obiang,Pedro Mba Obiang Avomo,"CDM, CM",77.0,80.0,9000000.0,60000.0,...,67.0,59.0,74.0,74.0,75.0,76.0,68.0,50.0,8,2
2865,200054.0,19,1.0,Pedro Obiang,Pedro Mba Obiang Avomo,"CDM, CM",77.0,80.0,9000000.0,60000.0,...,67.0,59.0,74.0,74.0,75.0,76.0,68.0,50.0,0,1
3540,200054.0,18,1.0,Pedro Obiang,Pedro Mba Obiang Avomo,"CDM, CM",77.0,82.0,10000000.0,80000.0,...,68.0,59.0,75.0,74.0,75.0,77.0,68.0,50.0,4,2
3541,200054.0,18,1.0,Pedro Obiang,Pedro Mba Obiang Avomo,"CDM, CM",77.0,82.0,10000000.0,80000.0,...,68.0,59.0,75.0,74.0,75.0,77.0,68.0,50.0,2,0
4254,200054.0,17,1.0,Pedro Obiang,Pedro Mba Obiang Avomo,"CDM, CM",76.0,80.0,6000000.0,80000.0,...,68.0,59.0,74.0,72.0,74.0,76.0,68.0,50.0,9,9
4255,200054.0,17,1.0,Pedro Obiang,Pedro Mba Obiang Avomo,"CDM, CM",76.0,80.0,6000000.0,80000.0,...,68.0,59.0,74.0,72.0,74.0,76.0,68.0,50.0,1,1
4962,200054.0,16,1.0,Pedro Obiang,Pedro Mba Obiang Avomo,"CM, CDM",74.0,81.0,4700000.0,35000.0,...,68.0,59.0,74.0,72.0,73.0,77.0,68.0,50.0,7,2
4963,200054.0,16,1.0,Pedro Obiang,Pedro Mba Obiang Avomo,"CM, CDM",74.0,81.0,4700000.0,35000.0,...,68.0,59.0,74.0,72.0,73.0,77.0,68.0,50.0,0,1


In [894]:
df = df.drop([4692, 4254, 3540, 2864])

In [895]:
df[df['player_id']==189505.0][['player_id', 'short_name','long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
2037,189505.0,Pedro,Pedro Eliezer Rodríguez Ledesma,20,1,1
2704,189505.0,Pedro,Pedro Eliezer Rodríguez Ledesma,19,8,2
3334,189505.0,Pedro,Pedro Eliezer Rodríguez Ledesma,18,4,2
4017,189505.0,Pedro,Pedro Eliezer Rodríguez Ledesma,17,9,9
4657,189505.0,Pedro,Pedro Eliezer Rodríguez Ledesma,16,7,2


Pedro  messed  with  Pedro  Obiang's  stats  and  other  players  below

In [896]:
df[df['player_id']==188377.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
29,188377.0,Kyle Walker,23,1,0
679,188377.0,Kyle Walker,22,0,2
680,188377.0,Kyle Walker,22,1,2
1336,188377.0,Kyle Walker,21,1,1
1337,188377.0,Kyle Walker,21,0,3
2005,188377.0,Kyle Walker,20,1,4
2671,188377.0,Kyle Walker,19,1,1
2672,188377.0,Kyle Walker,19,0,3
3348,188377.0,Kyle Walker,18,0,6
3349,188377.0,Kyle Walker,18,0,2


In [897]:
df = df.drop([3349, 2672, 1337, 680])
df.loc[29, ['goals', 'assists']] = [0, 0]

In [898]:
df[df['player_id']==227927.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
191,227927.0,Kyle Walker-Peters,23,1,0
876,227927.0,Kyle Walker-Peters,22,0,2
877,227927.0,Kyle Walker-Peters,22,1,2
1627,227927.0,Kyle Walker-Peters,21,1,1
1628,227927.0,Kyle Walker-Peters,21,0,3
2362,227927.0,Kyle Walker-Peters,20,1,4
3138,227927.0,Kyle Walker-Peters,19,1,1
3139,227927.0,Kyle Walker-Peters,19,0,3
3832,227927.0,Kyle Walker-Peters,18,0,6
3833,227927.0,Kyle Walker-Peters,18,0,2


In [899]:
df = df.drop([3832, 3138, 1627, 876])
df.loc[2362, ['goals', 'assists']] = [0, 0] #Attributed Kyle Walker's Stats
df.loc[4570, ['goals', 'assists']] = [0, 0] # Attributed Kyle Walker's Stats

In [900]:
df[df['player_id']==200778.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
1738,200778.0,Cyrus Sylvester Frederick Christie,21,1,0
3061,200778.0,Cyrus Sylvester Frederick Christie,19,0,1
3062,200778.0,Cyrus Sylvester Frederick Christie,19,1,1


In [901]:
df = df.drop([3062])
df.loc[1738, ['goals', 'assists']] = [0, 0]

In [902]:
df[df['player_id']==206083.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
3130,206083.0,Josh Murphy,19,0,1
3131,206083.0,Josh Murphy,19,3,2


In [903]:
df = df.drop(3130)

In [904]:
df[df['player_id']==206085.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
408,206085.0,Jacob Murphy,23,4,2
1078,206085.0,Jacob Murphy,22,1,2
1741,206085.0,Jacob Murphy,21,2,3
3067,206085.0,Jacob Murphy,19,0,1
3068,206085.0,Jacob Murphy,19,3,2
3796,206085.0,Jacob Murphy,18,1,1


In [905]:
df = df.drop(3068)

In [906]:
df[df['player_id']==232805.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
1672,232805.0,Bernardo Fernandes da Silva Junior,21,1,0
2316,232805.0,Bernardo Fernandes da Silva Junior,20,3,2
2978,232805.0,Bernardo Fernandes da Silva Junior,19,1,4
2979,232805.0,Bernardo Fernandes da Silva Junior,19,0,1


In [907]:
df = df.drop(2978)
df.loc[2316, ['goals', 'assists']] = [0, 0]
df.loc[1672, ['goals', 'assists']] = [0, 0]

In [908]:
df[df['player_id']==218667.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
10,218667.0,Bernardo Mota Veiga de Carvalho e Silva,23,4,6
672,218667.0,Bernardo Mota Veiga de Carvalho e Silva,22,8,4
1327,218667.0,Bernardo Mota Veiga de Carvalho e Silva,21,1,0
1328,218667.0,Bernardo Mota Veiga de Carvalho e Silva,21,2,6
1991,218667.0,Bernardo Mota Veiga de Carvalho e Silva,20,3,2
1992,218667.0,Bernardo Mota Veiga de Carvalho e Silva,20,6,7
2679,218667.0,Bernardo Mota Veiga de Carvalho e Silva,19,1,4
2680,218667.0,Bernardo Mota Veiga de Carvalho e Silva,19,0,1
2681,218667.0,Bernardo Mota Veiga de Carvalho e Silva,19,7,7
3340,218667.0,Bernardo Mota Veiga de Carvalho e Silva,18,6,4


In [909]:
df[(df['fifa_version'] == 19) & (df['goals'] == 7)][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
2661,204485.0,Riyad Mahrez,19,7,4
2681,218667.0,Bernardo Mota Veiga de Carvalho e Silva,19,7,7
2693,230666.0,Gabriel Fernando de Jesus,19,7,3
2784,178224.0,Javier Hernández Balcázar,19,7,1
2869,205569.0,James Ward-Prowse,19,7,0
2905,195859.0,Daniel William John Ings,19,7,3
2955,194138.0,Andre Gray,19,7,2
2963,207807.0,Ryan Fraser,19,7,14
2971,220697.0,James Maddison,19,7,7
3135,220196.0,David Robert Brooks,19,7,5


Checking  to  see  if  Bernardo  Silva's  stats  were  attributed  to  someone  else.  This  shows  they  weren't  since  James  Maddison  actually achieved  7  goals  and  7  assists.

In [910]:
#Dropping some of Bernardo Silva's Stats
df = df.drop([2680, 2679, 1991, 1327])

In [911]:
df[df['player_id']==205525.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
1475,205525.0,Bernard Anício Caldeira Duarte,21,1,0
2084,205525.0,Bernard Anício Caldeira Duarte,20,3,2


These  stats  are  correct

In [912]:
df[df['player_id']==238616.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
197,238616.0,Pedro Lomba Neto,23,0,1
847,238616.0,Pedro Lomba Neto,22,1,1
1729,238616.0,Pedro Lomba Neto,21,5,6
2508,238616.0,Pedro Lomba Neto,20,1,1
2509,238616.0,Pedro Lomba Neto,20,3,3


In [913]:
df = df.drop(2508)

In [914]:
df[df['player_id']==205346.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
312,205346.0,Ryan Fredericks,23,2,2
910,205346.0,Ryan Fredericks,22,4,5
1564,205346.0,Ryan Fredericks,21,1,0
1565,205346.0,Ryan Fredericks,21,1,1
2341,205346.0,Ryan Fredericks,20,0,3
3036,205346.0,Ryan Fredericks,19,1,1
3037,205346.0,Ryan Fredericks,19,1,0


In [915]:
df = df.drop([1564,3036])

In [916]:
df[df['player_id']==226790.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
44,226790.0,Onyinye Wilfred Ndidi,23,2,2
684,226790.0,Onyinye Wilfred Ndidi,22,4,5
1353,226790.0,Onyinye Wilfred Ndidi,21,1,0
1354,226790.0,Onyinye Wilfred Ndidi,21,1,4
2063,226790.0,Onyinye Wilfred Ndidi,20,2,1
2771,226790.0,Onyinye Wilfred Ndidi,19,1,1
2772,226790.0,Onyinye Wilfred Ndidi,19,2,0
3553,226790.0,Onyinye Wilfred Ndidi,18,0,3


In [917]:
df = df.drop([2771,1353])
df.loc[44, ['goals', 'assists']] = [0,0]
df.loc[684, ['goals', 'assists']] = [0,0]

In [918]:
df[df['player_id']==200949.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
104,200949.0,Lucas Rodrigues Moura da Silva,23,1,0
105,200949.0,Lucas Rodrigues Moura da Silva,23,2,6
738,200949.0,Lucas Rodrigues Moura da Silva,22,2,6
739,200949.0,Lucas Rodrigues Moura da Silva,22,7,2
1360,200949.0,Lucas Rodrigues Moura da Silva,21,3,4
1361,200949.0,Lucas Rodrigues Moura da Silva,21,2,2
2020,200949.0,Lucas Rodrigues Moura da Silva,20,4,4
2021,200949.0,Lucas Rodrigues Moura da Silva,20,3,2
2709,200949.0,Lucas Rodrigues Moura da Silva,19,10,1


In [919]:
df = df.drop([2021,1361,739,105])

In [920]:
df[df['player_id']==231866.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
19,231866.0,Rodrigo Hernández Cascante,23,2,6
20,231866.0,Rodrigo Hernández Cascante,23,13,1
673,231866.0,Rodrigo Hernández Cascante,22,7,2
674,231866.0,Rodrigo Hernández Cascante,22,6,0
1344,231866.0,Rodrigo Hernández Cascante,21,2,2
1345,231866.0,Rodrigo Hernández Cascante,21,7,2
1998,231866.0,Rodrigo Hernández Cascante,20,3,2


In [921]:
df = df.drop([1345,674,20])

In [922]:
df[df['player_id']==198329.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
207,198329.0,Rodrigo Moreno Machado,23,2,6
208,198329.0,Rodrigo Moreno Machado,23,13,1
786,198329.0,Rodrigo Moreno Machado,22,7,2
787,198329.0,Rodrigo Moreno Machado,22,6,0


In [923]:
df = df.drop([786,207])

In [924]:
df[df['player_id']==209297.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
110,209297.0,Frederico Rodrigues de Paula Santos,23,2,2
111,209297.0,Frederico Rodrigues de Paula Santos,23,2,6
743,209297.0,Frederico Rodrigues de Paula Santos,22,4,5
744,209297.0,Frederico Rodrigues de Paula Santos,22,7,2
1389,209297.0,Frederico Rodrigues de Paula Santos,21,1,0
1390,209297.0,Frederico Rodrigues de Paula Santos,21,2,2
2119,209297.0,Frederico Rodrigues de Paula Santos,20,3,2
2715,209297.0,Frederico Rodrigues de Paula Santos,19,1,1


In [925]:
df = df.drop([1390,744,111])

In [926]:
df[(df['fifa_version']==22) & (df['goals']==7)][['player_id','fifa_version','short_name','long_name', 'club_name', 'goals', 'assists']]

,player_id,fifa_version,short_name,long_name,club_name,goals,assists
673,231866.0,22,Rodri,Rodrigo Hernández Cascante,Manchester City,7,2
736,195859.0,22,D. Ings,Daniel William John Ings,Aston Villa,7,6
924,226093.0,22,C. Adams,Ché Adams,Southampton,7,3
983,234824.0,22,Y. Wissa,Yoane Wissa,Brentford,7,0
994,169792.0,22,J. Rodriguez,Jay Rodriguez,Burnley,7,2
1175,259184.0,22,R. Vilca,Rodrigo Gary Vilca Betetta,Newcastle United,7,2


This  shows  that  Rodri  was  messing  with  Fred's  stats

In [927]:
df[df['player_id']==259184.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
1175,259184.0,Rodrigo Gary Vilca Betetta,22,7,2
1176,259184.0,Rodrigo Gary Vilca Betetta,22,6,0


In [928]:
df.loc[1175, ['goals', 'assists']] = [0,0]
df.loc[1176, ['goals', 'assists']] = [0,0]

In [929]:
df[df['player_id']==169792.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
994,169792.0,Jay Rodriguez,22,7,2
1546,169792.0,Jay Rodriguez,21,1,2
2217,169792.0,Jay Rodriguez,20,8,2
3562,169792.0,Jay Rodriguez,18,7,1
4192,169792.0,Jay Rodriguez,17,5,2
4803,169792.0,Jay Rodriguez,16,0,0
5508,169792.0,Jay Rodriguez,15,0,0


In [930]:
# Weirdly, only the 22' season was messed up
df.loc[1175, ['goals', 'assists']] = [2,2]

In [931]:
df[df['player_id']==206518.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
1720,206518.0,Callum Jack Robinson,21,5,0
1721,206518.0,Callum Jack Robinson,21,0,2
2395,206518.0,Callum Jack Robinson,20,1,2
5837,206518.0,Callum Jack Robinson,15,0,0


In [932]:
df = df.drop(1721)

In [933]:
df[(df['fifa_version']==21) & (df['goals']==0) & (df['assists']==2)][['player_id','fifa_version','short_name','long_name', 'club_name', 'goals', 'assists']]


,player_id,fifa_version,short_name,long_name,club_name,goals,assists
1318,215914.0,21,N. Kanté,N'Golo Kanté,Chelsea,0,2
1346,135507.0,21,Fernandinho,Fernando Luiz Rosa,Manchester City,0,2
1381,193348.0,21,X. Shaqiri,Xherdan Shaqiri,Liverpool,0,2
1382,201417.0,21,M. Doherty,Matthew James Doherty,Wolverhampton Wanderers,0,2
1418,205923.0,21,B. Davies,Benjamin Thomas Davies,Tottenham Hotspur,0,2
1506,189177.0,21,J. Fleck,John Fleck,Sheffield United,0,2
1517,200855.0,21,G. Baldock,George Baldock,Sheffield United,0,2
1527,212491.0,21,A. Masuaku,Arthur Masuaku,West Ham United,0,2
1561,200746.0,21,J. Lundstram,John David Lundstram,Sheffield United,0,2
1573,218659.0,21,M. Targett,Matt Targett,Aston Villa,0,2


Jack  Robinson  was  the  cause  for  Callum  Jack  Robinson's  issue

In [934]:
df[df['player_id']==199027.0][['player_id', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,long_name,fifa_version,goals,assists
1767,199027.0,Jack Robinson,21,0,2


In [935]:
df[df['player_id']==210514.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
9,210514.0,João Cancelo,João Pedro Cavaco Cancelo,23,2,2
669,210514.0,João Cancelo,João Pedro Cavaco Cancelo,22,1,7
670,210514.0,João Cancelo,João Pedro Cavaco Cancelo,22,3,0
1364,210514.0,João Cancelo,João Pedro Cavaco Cancelo,21,2,3
2010,210514.0,João Cancelo,João Pedro Cavaco Cancelo,20,1,1


In [936]:
df = df.drop(670)
df.loc[2010, ['goals', 'assists']] = [0,0]

In [937]:
df[(df['fifa_version']==22) & (df['goals']==3) & (df['assists']==0)][['player_id','fifa_version','short_name','long_name', 'club_name', 'goals', 'assists']]


,player_id,fifa_version,short_name,long_name,club_name,goals,assists
676,164240.0,22,Thiago Silva,Thiago Emiliano da Silva,Chelsea,3,0
779,189682.0,22,B. Mee,Ben Mee,Burnley,3,0
867,212602.0,22,Diego Llorente,Diego Javier Llorente Ríos,Leeds United,3,0
981,226627.0,22,T. Minamino,南野 拓実,Liverpool,3,0
1020,224494.0,22,R. Henry,Rico Henry,Brentford,3,0
1108,252042.0,22,João Pedro,João Pedro Junqueira de Jesus,Watford,3,0


In [938]:
df[df['player_id']==252042.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
1108,252042.0,João Pedro,João Pedro Junqueira de Jesus,22,3,0


No  errors  here

In [939]:
df[df['player_id']==180819.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
297,180819.0,A. Lallana,Adam David Lallana,23,2,1
942,180819.0,A. Lallana,Adam David Lallana,22,0,1
943,180819.0,A. Lallana,Adam David Lallana,22,0,2
1495,180819.0,A. Lallana,Adam David Lallana,21,1,1
2100,180819.0,A. Lallana,Adam David Lallana,20,1,1
2722,180819.0,A. Lallana,Adam David Lallana,19,0,0
3347,180819.0,A. Lallana,Adam David Lallana,18,0,0
4028,180819.0,A. Lallana,Adam David Lallana,17,8,7
4709,180819.0,A. Lallana,Adam David Lallana,16,4,6
5389,180819.0,A. Lallana,Adam David Lallana,15,5,3


In [940]:
df = df.drop(943)

In [941]:
df[df['player_id']==216388.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
83,216388.0,A. Saint-Maximin,Allan Saint-Maximin,23,1,5
800,216388.0,A. Saint-Maximin,Allan Saint-Maximin,22,0,2
801,216388.0,A. Saint-Maximin,Allan Saint-Maximin,22,5,5
1421,216388.0,A. Saint-Maximin,Allan Saint-Maximin,21,3,4
2127,216388.0,A. Saint-Maximin,Allan Saint-Maximin,20,3,5


In [942]:
df = df.drop(800)

In [943]:
df[df['short_name'] == 'Allan' ][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
102,199914.0,Allan,Allan Marques Loureiro,23,0,0
703,199914.0,Allan,Allan Marques Loureiro,22,0,2


All  Good

In [944]:
df[df['player_id']==247851.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
99,247851.0,Bruno Guimarães,Bruno Guimarães Rodriguez Moura,23,4,5
100,247851.0,Bruno Guimarães,Bruno Guimarães Rodriguez Moura,23,2,6


In [945]:
df = df.drop(100)

In [946]:
df[(df['fifa_version']==23) & (df['goals']==2) & (df['assists']==6)][['player_id','fifa_version','short_name','long_name', 'club_name', 'goals', 'assists']]

,player_id,fifa_version,short_name,long_name,club_name,goals,assists
19,231866.0,23,Rodri,Rodrigo Hernández Cascante,Manchester City,2,6
157,227535.0,23,R. Bentancur,Rodrigo Bentancur Colmán,Tottenham Hotspur,2,6
484,264337.0,23,Rodrigo Muniz,Rodrigo Muniz Carvalho,Fulham,2,6


It  is  clear  from  this  that  Rodri  has  affected  two  other  players  as well

In [947]:
df[df['player_id']==227535.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
157,227535.0,R. Bentancur,Rodrigo Bentancur Colmán,23,2,6
158,227535.0,R. Bentancur,Rodrigo Bentancur Colmán,23,13,1
159,227535.0,R. Bentancur,Rodrigo Bentancur Colmán,23,5,2


In [948]:
df = df.drop([158,157])

In [949]:
df[df['player_id']==264337.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
484,264337.0,Rodrigo Muniz,Rodrigo Muniz Carvalho,23,2,6
485,264337.0,Rodrigo Muniz,Rodrigo Muniz Carvalho,23,13,1


In [950]:
df = df.drop(485)
df.loc[484, ['goals', 'assists']] = [0,0]

In [951]:
df[df['player_id']==247204.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
201,247204.0,Emerson Royal,Emerson Aparecido Leite de Souza Junior,23,1,0
202,247204.0,Emerson Royal,Emerson Aparecido Leite de Souza Junior,23,2,1


In [952]:
df = df.drop(201)

In [953]:
df[(df['fifa_version']==23) & (df['goals']==1) & (df['assists']==0) & (df['club_name'].notna())][['player_id','fifa_version','short_name','long_name', 'club_name', 'goals', 'assists']]

,player_id,fifa_version,short_name,long_name,club_name,goals,assists
1,20801.0,23,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,Manchester United,1,0
36,197061.0,23,J. Matip,Joël Andre Job Matip,Liverpool,1,0
63,189242.0,23,Coutinho,Philippe Coutinho Correia,Aston Villa,1,0
66,200458.0,23,L. Digne,Lucas Digne,Aston Villa,1,0
97,239301.0,23,L. Martínez,Lisandro Martínez,Manchester United,1,0
104,200949.0,23,Lucas Moura,Lucas Rodrigues Moura da Silva,Tottenham Hotspur,1,0
125,237238.0,23,S. McTominay,Scott McTominay,Manchester United,1,0
134,188988.0,23,M. Lanzini,Manuel Lanzini,West Ham United,1,0
141,199915.0,23,L. Dunk,Lewis Dunk,Brighton & Hove Albion,1,0
149,210455.0,23,Jonny,Jonathan Castro Otto,Wolverhampton Wanderers,1,0


In [954]:
df[df['player_id']==210736.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
214,210736.0,Emerson,Emerson Palmieri dos Santos,23,1,0
830,210736.0,Emerson,Emerson Palmieri dos Santos,22,0,0
1444,210736.0,Emerson,Emerson Palmieri dos Santos,21,0,0
2200,210736.0,Emerson,Emerson Palmieri dos Santos,20,0,0
2913,210736.0,Emerson,Emerson Palmieri dos Santos,19,0,0


In [955]:
df[df['player_id']==207948.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
254,207948.0,B. Traoré,Bertrand Isidore Traoré,23,2,0
255,207948.0,B. Traoré,Bertrand Isidore Traoré,23,0,1
860,207948.0,B. Traoré,Bertrand Isidore Traoré,22,0,0
4972,207948.0,B. Traoré,Bertrand Isidore Traoré,16,2,1


In [956]:
df = df.drop(255)

In [957]:
df[df['long_name'].str.lower().str.contains('fred')][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
44,226790.0,W. Ndidi,Onyinye Wilfred Ndidi,Leicester City,23,0,0
110,209297.0,Fred,Frederico Rodrigues de Paula Santos,Manchester United,23,2,2
312,205346.0,R. Fredericks,Ryan Fredericks,AFC Bournemouth,23,2,2
466,139062.0,W. Caballero,Wilfredo Daniel Caballero,Southampton,23,2,2
486,190311.0,L. Taylor,Lyle James Alfred Taylor,Nottingham Forest,23,2,2
633,264579.0,F. Potts,Freddie Potts,West Ham United,23,2,2
684,226790.0,W. Ndidi,Onyinye Wilfred Ndidi,Leicester City,22,0,0
743,209297.0,Fred,Frederico Rodrigues de Paula Santos,Manchester United,22,4,5
910,205346.0,R. Fredericks,Ryan Fredericks,West Ham United,22,4,5
1017,222514.0,F. Woodman,Freddie Woodman,Newcastle United,22,4,5


In [958]:
df.loc[2843, ['goals', 'assists']] = [0,0]
df.loc[2843, ['goals', 'assists']] = [0,0]
df.loc[1585, ['goals', 'assists']] = [0,0]
df.loc[1725, ['goals', 'assists']] = [0,0]
df.loc[1228, ['goals', 'assists']] = [0,0]
df.loc[1017, ['goals', 'assists']] = [0,0]
df.loc[910, ['goals', 'assists']] = [0,0]
df.loc[633, ['goals', 'assists']] = [0,0]
df.loc[486, ['goals', 'assists']] = [0,0]
df.loc[466, ['goals', 'assists']] = [0,0]
df.loc[312, ['goals', 'assists']] = [0,0]

In [959]:
df[df['long_name'].str.lower().str.contains('rodri')][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
19,231866.0,Rodri,Rodrigo Hernández Cascante,Manchester City,23,2,6
99,247851.0,Bruno Guimarães,Bruno Guimarães Rodriguez Moura,Newcastle United,23,4,5
104,200949.0,Lucas Moura,Lucas Rodrigues Moura da Silva,Tottenham Hotspur,23,1,0
110,209297.0,Fred,Frederico Rodrigues de Paula Santos,Manchester United,23,2,2
159,227535.0,R. Bentancur,Rodrigo Bentancur Colmán,Tottenham Hotspur,23,5,2
208,198329.0,Rodrigo,Rodrigo Moreno Machado,Leeds United,23,13,1
484,264337.0,Rodrigo Muniz,Rodrigo Muniz Carvalho,Fulham,23,0,0
673,231866.0,Rodri,Rodrigo Hernández Cascante,Manchester City,22,7,2
738,200949.0,Lucas Moura,Lucas Rodrigues Moura da Silva,Tottenham Hotspur,22,2,6
743,209297.0,Fred,Frederico Rodrigues de Paula Santos,Manchester United,22,4,5


In [960]:
df.loc[2119, ['goals', 'assists']] = [0,0]

In [961]:
df[df['long_name'].str.lower().str.contains('allan')][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
83,216388.0,A. Saint-Maximin,Allan Saint-Maximin,Newcastle United,23,1,5
102,199914.0,Allan,Allan Marques Loureiro,Everton,23,0,0
297,180819.0,A. Lallana,Adam David Lallana,Brighton & Hove Albion,23,2,1
703,199914.0,Allan,Allan Marques Loureiro,Everton,22,0,2
801,216388.0,A. Saint-Maximin,Allan Saint-Maximin,Newcastle United,22,5,5
942,180819.0,A. Lallana,Adam David Lallana,Brighton & Hove Albion,22,0,1
1291,244276.0,T. Allan,Thomas David Allan,Newcastle United,22,0,2
1297,248711.0,A. Tchaptchet,Allan Tchaptchet,Southampton,22,0,2
1421,216388.0,A. Saint-Maximin,Allan Saint-Maximin,Newcastle United,21,3,4
1495,180819.0,A. Lallana,Adam David Lallana,Brighton & Hove Albion,21,1,1


In [962]:
df[(df['short_name'].str.match(r'^[A-Za-z]+$', na=False)) & ((df['goals'] > 0) | (df['assists'] > 0))][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
6,210257.0,Ederson,Ederson Santana de Moraes,Manchester City,23,0,1
7,212831.0,Alisson,Alisson Ramsés Becker,Liverpool,23,0,1
16,209499.0,Fabinho,Fábio Henrique Tavares,Liverpool,23,0,2
19,231866.0,Rodri,Rodrigo Hernández Cascante,Manchester City,23,2,6
30,205498.0,Jorginho,Luiz Frello Filho Jorge,Chelsea,23,2,1
...,...,...,...,...,...,...,...
5393,190782.0,Sandro,Sandro Raniere Guimarães Cordeiro,Tottenham Hotspur,15,1,0
5412,172672.0,Varela,Silvestre Manuel Gonçalves Varela,West Bromwich Albion,15,1,0
5438,176993.0,Bojan,Bojan Krkíc Pérez,Stoke City,15,4,1
5447,189681.0,Rafael,Rafael Pereira da Silva,Manchester United,15,0,1


In [963]:
df[(df['short_name'].str.match(r'^[A-Za-z]+$', na=False)) & ((df['goals'] > 0) | (df['assists'] > 0))].drop_duplicates(subset='player_id')[['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
6,210257.0,Ederson,Ederson Santana de Moraes,Manchester City,23,0,1
7,212831.0,Alisson,Alisson Ramsés Becker,Liverpool,23,0,1
16,209499.0,Fabinho,Fábio Henrique Tavares,Liverpool,23,0,2
19,231866.0,Rodri,Rodrigo Hernández Cascante,Manchester City,23,2,6
30,205498.0,Jorginho,Luiz Frello Filho Jorge,Chelsea,23,2,1
63,189242.0,Coutinho,Philippe Coutinho Correia,Aston Villa,23,1,0
91,231943.0,Richarlison,Richarlison de Andrade,Tottenham Hotspur,23,1,4
92,232580.0,Gabriel,Gabriel dos Santos Magalhães,Arsenal,23,3,0
96,239231.0,Cucurella,Marc Cucurella Saseta,Chelsea,23,0,2
110,209297.0,Fred,Frederico Rodrigues de Paula Santos,Manchester United,23,2,2


In [964]:
df[df['long_name'].str.lower().str.contains('gabriel')][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
59,230666.0,Gabriel Jesus,Gabriel Fernando de Jesus,Arsenal,23,11,7
60,232488.0,C. Romero,Cristian Gabriel Romero,Tottenham Hotspur,23,0,1
75,253072.0,D. Núñez,Darwin Gabriel Núñez Ribeiro,Liverpool,23,9,3
92,232580.0,Gabriel,Gabriel dos Santos Magalhães,Arsenal,23,3,0
127,243715.0,W. Saliba,William Alain André Gabriel Saliba,Arsenal,23,2,1
203,251566.0,Gabriel Martinelli,Gabriel Teodoro Martinelli Silva,Arsenal,23,15,6
713,230666.0,Gabriel Jesus,Gabriel Fernando de Jesus,Manchester City,22,8,9
730,232488.0,C. Romero,Cristian Gabriel Romero,Tottenham Hotspur,22,0,0
809,232580.0,Gabriel,Gabriel dos Santos Magalhães,Arsenal,22,5,0
935,251566.0,Gabriel Martinelli,Gabriel Teodoro Martinelli Silva,Arsenal,22,6,6


In [965]:
df[df['long_name'].str.lower().str.contains('thiago')][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
21,164240.0,Thiago Silva,Thiago Emiliano da Silva,Chelsea,23,0,2
22,189509.0,Thiago,Thiago Alcântara do Nascimento,Liverpool,23,0,0
664,189509.0,Thiago,Thiago Alcântara do Nascimento,Liverpool,22,1,4
676,164240.0,Thiago Silva,Thiago Emiliano da Silva,Chelsea,22,3,0


In [966]:
df[df['long_name'].str.lower().str.contains('kenedy')][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
266,215639.0,Kenedy,Robert Kenedy Nunes do Nascimento,Chelsea,23,0,0
835,215639.0,Kenedy,Robert Kenedy Nunes do Nascimento,Chelsea,22,0,1
1571,215639.0,Kenedy,Robert Kenedy Nunes do Nascimento,Chelsea,21,0,0
2306,215639.0,Kenedy,Robert Kenedy Nunes do Nascimento,Chelsea,20,0,0
2920,215639.0,Kenedy,Robert Kenedy Nunes do Nascimento,Newcastle United,19,1,1
3724,215639.0,Kenedy,Robert Kenedy Nunes do Nascimento,Chelsea,18,2,2
4352,215639.0,Kenedy,Robert Kenedy Nunes do Nascimento,Chelsea,17,0,0
5170,215639.0,Kenedy,Robert Kenedy Nunes do Nascimento,Chelsea,16,1,0


In [967]:
df[df['long_name'].str.lower().str.contains('pedro')][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
9,210514.0,João Cancelo,João Pedro Cavaco Cancelo,Manchester City,23,2,2
82,212442.0,José Sá,José Pedro Malheiro de Sá,Wolverhampton Wanderers,23,0,0
197,238616.0,Pedro Neto,Pedro Lomba Neto,Wolverhampton Wanderers,23,0,1
669,210514.0,João Cancelo,João Pedro Cavaco Cancelo,Manchester City,22,1,7
798,212442.0,José Sá,José Pedro Malheiro de Sá,Wolverhampton Wanderers,22,0,1
847,238616.0,Pedro Neto,Pedro Lomba Neto,Wolverhampton Wanderers,22,1,1
1108,252042.0,João Pedro,João Pedro Junqueira de Jesus,Watford,22,3,0
1348,178005.0,Rui Patrício,Rui Pedro dos Santos Patrício,Wolverhampton Wanderers,21,0,0
1364,210514.0,João Cancelo,João Pedro Cavaco Cancelo,Manchester City,21,2,3
1729,238616.0,Pedro Neto,Pedro Lomba Neto,Wolverhampton Wanderers,21,5,6


In [968]:
df = df.drop(4962)

In [969]:
df[df['player_id']==178005.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
1348,178005.0,Rui Patrício,Rui Pedro dos Santos Patrício,21,0,0
2013,178005.0,Rui Patrício,Rui Pedro dos Santos Patrício,20,1,1
2683,178005.0,Rui Patrício,Rui Pedro dos Santos Patrício,19,8,2


In [970]:
df.loc[1348, ['goals', 'assists']] = [0,0]
df.loc[2013, ['goals', 'assists']] = [0,0]
df.loc[2683, ['goals', 'assists']] = [0,0]

In [971]:
df[df['player_id']==240950.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
3274,240950.0,Pedro Gonçalves,Pedro António Pereira Gonçalves,19,8,2


In [972]:
df.loc[3274, ['goals', 'assists']] = [0,0]

In [973]:
df[df['player_id']==183518.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
3052,183518.0,Rui Fonte,Rui Pedro da Rocha Fonte,19,8,2


In [974]:
df.loc[3052, ['goals', 'assists']] = [0,0]

In [975]:
df[df['player_id']==224019.0][['player_id', 'short_name', 'long_name',  'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,fifa_version,goals,assists
2506,224019.0,Pedro Chirivella,Pedro Chirivella Burgos,20,1,1
4564,224019.0,Pedro Chirivella,Pedro Chirivella Burgos,17,9,9


In [976]:
df.loc[2506, ['goals', 'assists']] = [0,0]
df.loc[4564, ['goals', 'assists']] = [0,0]

In [977]:
df.loc[798, ['goals', 'assists']] = [0,0]

In [978]:
df[df['long_name'].str.lower().str.contains('wesley')][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
168,248695.0,W. Fofana,Wesley Fofana,Leicester City,23,1,0
851,248695.0,W. Fofana,Wesley Fofana,Leicester City,22,0,0
929,232381.0,Wesley,Wesley Moraes Ferreira da Silva,Aston Villa,22,0,0
1580,232381.0,Wesley,Wesley Moraes Ferreira da Silva,Aston Villa,21,0,0
1620,216774.0,W. Hoedt,Wesley Theodorus Hoedt,Southampton,21,0,0
1734,193182.0,W. Foderingham,Wesley Foderingham,Sheffield United,21,0,0
2135,232381.0,Wesley,Wesley Moraes Ferreira da Silva,Aston Villa,20,5,1
2308,216774.0,W. Hoedt,Wesley Theodorus Hoedt,Southampton,20,0,0
2922,216774.0,W. Hoedt,Wesley Theodorus Hoedt,Southampton,19,0,0
4884,149791.0,W. Hoolahan,Wesley Hoolahan,Norwich City,16,4,8


In [979]:
df[df['long_name'].str.lower().str.contains('bernardo')][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
10,218667.0,Bernardo Silva,Bernardo Mota Veiga de Carvalho e Silva,Manchester City,23,4,6
672,218667.0,Bernardo Silva,Bernardo Mota Veiga de Carvalho e Silva,Manchester City,22,8,4
1299,247622.0,Bernardo Rosa,Bernardo Costa da Rosa,West Ham United,22,0,0
1328,218667.0,Bernardo Silva,Bernardo Mota Veiga de Carvalho e Silva,Manchester City,21,2,6
1672,232805.0,Bernardo,Bernardo Fernandes da Silva Junior,Brighton & Hove Albion,21,0,0
1956,247622.0,Bernardo Rosa,Bernardo Costa da Rosa,West Ham United,21,1,0
1992,218667.0,Bernardo Silva,Bernardo Mota Veiga de Carvalho e Silva,Manchester City,20,6,7
2316,232805.0,Bernardo,Bernardo Fernandes da Silva Junior,Brighton & Hove Albion,20,0,0
2681,218667.0,Bernardo Silva,Bernardo Mota Veiga de Carvalho e Silva,Manchester City,19,7,7
2979,232805.0,Bernardo,Bernardo Fernandes da Silva Junior,Brighton & Hove Albion,19,0,1


In [980]:
df[df['long_name'].str.lower().str.contains('oscar')][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
4016,188152.0,Oscar,Oscar dos Santos Emboaba Júnior,Chelsea,17,0,1
4645,188152.0,Oscar,Oscar dos Santos Emboaba Júnior,Chelsea,16,3,3
5247,230828.0,O. Borg,Oscar Francis Borg,West Ham United,16,3,3
5305,188152.0,Oscar,Oscar dos Santos Emboaba Júnior,Chelsea,15,6,8


In [981]:
df.loc[5247, ['goals', 'assists']] = [0,0]

In [982]:
df[df['long_name'].str.lower().str.contains('sokratis')][['player_id', 'short_name', 'long_name', 'club_name', 'fifa_version', 'goals', 'assists']]

,player_id,short_name,long_name,club_name,fifa_version,goals,assists
1430,172879.0,Sokratis,Sokratis Papastathopoulos,Arsenal,21,0,0
2000,172879.0,Sokratis,Sokratis Papastathopoulos,Arsenal,20,2,0
2668,172879.0,Sokratis,Sokratis Papastathopoulos,Arsenal,19,1,2


In [983]:
df.columns

Index(['player_id', 'fifa_version', 'fifa_update', 'short_name', 'long_name',
       'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age', 'dob', 'height_cm', 'weight_kg', 'league_id', 'league_name',
       'league_level', 'club_team_id', 'club_name', 'club_position',
       'club_jersey_number', 'club_contract_valid_until_year',
       'nationality_name', 'nation_jersey_number', 'preferred_foot',
       'weak_foot', 'skill_moves', 'international_reputation', 'work_rate',
       'body_type', 'release_clause_eur', 'pace', 'shooting', 'passing',
       'dribbling', 'defending', 'physic', 'attacking_crossing',
       'attacking_finishing', 'goals', 'assists'],
      dtype='object')

In [984]:
pos_array = df['player_positions'].unique()
pos_array

array(['CM, CAM', 'ST', 'CB', 'RW', 'LW, LM', 'GK', 'CDM, CM', 'LB, RB',
       'CAM, CM', 'CDM, CB', 'LB', 'RB', 'CDM', 'CM', 'LW, RW', 'RW, RM',
       'CM, CDM', 'CF, LW', 'LW, CF, CAM', 'LM, LWB', 'RB, RM',
       'LW, LM, CAM', 'LW, RW, LM', 'CAM, RW', 'CAM, CF, ST', 'RWB, RB',
       'CF', 'RW, CAM', 'RB, RWB', 'ST, RW', 'CB, RWB, RB', 'CAM, LW, CM',
       'LW, CF', 'CAM', 'ST, LM, CF', 'LWB, LB', 'RM, LM', 'ST, LW', 'LM',
       'CB, RB', 'CM, CAM, ST', 'LW, ST', 'LB, CB, LM', 'CB, LB, CDM',
       'RW, CF', 'ST, LM', 'RM, ST', 'RM, RW, CAM', 'RM, RW, LW',
       'LM, LW', 'LM, CAM', 'CB, LWB, LB', 'LM, ST, CAM', 'LWB, RWB',
       'RW, LW', 'CM, CDM, CAM', 'RM', 'CAM, LM', 'CM, LB, RB', 'LB, CB',
       'CAM, CF, CM', 'CB, RB, RWB', 'CB, LB', 'LM, CF, LB', 'RB, LB',
       'LW, CAM, ST', 'RM, CM, CAM', 'LW, RW, ST', 'ST, RW, LW', 'CM, RM',
       'CAM, ST, CM', 'LB, LWB', 'LB, LM', 'LM, LW, CAM', 'LM, RM',
       'RM, ST, LM', 'LW, RW, RWB', 'CAM, RM', 'LM, CM', 'RM, CM',
    

In [985]:
len(pos_array)

408

In [986]:
df['preferred_foot'].unique()

array(['Right', 'Left'], dtype=object)

In [987]:
df['weak_foot'].unique()

array([5., 4., 3., 1., 2.])

In [988]:
df['work_rate'].unique()

array(['High/High', 'High/Low', 'Medium/High', 'High/Medium',
       'Medium/Medium', 'Low/High', 'Medium/Low', 'Low/Low', 'Low/Medium'],
      dtype=object)

In [989]:
df['release_clause_eur'].unique()

array([1.989e+08, 7.790e+07, 1.813e+08, 2.137e+08, 1.919e+08, 2.005e+08,
       1.694e+08, 1.521e+08, 1.332e+08, 1.526e+08, 1.804e+08, 3.034e+08,
       2.069e+08, 1.710e+07, 7.980e+07, 1.008e+08, 1.482e+08, 1.935e+08,
       1.704e+08, 2.220e+07, 1.027e+08, 1.341e+08, 1.045e+08, 1.550e+08,
       1.242e+08, 9.820e+07, 8.140e+07, 6.940e+07, 9.530e+07, 4.370e+07,
       1.463e+08, 2.245e+08, 5.040e+07, 5.800e+07, 6.380e+07, 7.700e+07,
       5.510e+07, 7.770e+07, 6.840e+07, 1.037e+08, 1.254e+08, 9.380e+07,
       1.215e+08, 1.107e+08, 1.732e+08, 1.097e+08, 1.078e+08, 5.270e+07,
       6.570e+07, 6.640e+07, 6.270e+07, 8.000e+07, 9.580e+07,       nan,
       6.500e+06, 3.700e+07, 5.610e+07, 5.700e+07, 5.230e+07, 4.850e+07,
       5.140e+07, 7.510e+07, 8.100e+07, 9.840e+07, 7.120e+07, 6.810e+07,
       1.271e+08, 1.261e+08, 2.000e+07, 3.950e+07, 2.600e+07, 4.740e+07,
       3.230e+07, 5.830e+07, 6.420e+07, 5.680e+07, 6.160e+07, 6.550e+07,
       6.220e+07, 6.520e+07, 5.630e+07, 7.310e+07, 

In [990]:
df['value_eur'].unique()

array([1.075e+08, 4.100e+07, 9.800e+07, 1.155e+08, 1.010e+08, 1.055e+08,
       8.800e+07, 7.900e+07, 7.200e+07, 8.250e+07, 9.750e+07, 1.480e+08,
       9.000e+06, 4.200e+07, 5.450e+07, 7.700e+07, 1.005e+08, 8.850e+07,
       1.200e+07, 5.550e+07, 7.250e+07, 5.650e+07, 7.850e+07, 6.450e+07,
       5.100e+07, 4.400e+07, 3.750e+07, 5.150e+07, 2.300e+07, 7.600e+07,
       1.095e+08, 2.650e+07, 3.050e+07, 3.450e+07, 3.900e+07, 2.900e+07,
       4.950e+07, 3.600e+07, 5.250e+07, 6.350e+07, 4.750e+07, 6.150e+07,
       5.750e+07, 5.200e+07, 8.450e+07, 5.700e+07, 5.600e+07, 2.850e+07,
       3.550e+07, 3.300e+07, 4.050e+07, 4.850e+07, 3.400e+06, 2.000e+07,
       2.950e+07, 3.000e+07, 2.750e+07, 2.550e+07, 2.600e+07, 3.800e+07,
       4.800e+07, 3.700e+07, 6.050e+07, 1.050e+07, 1.300e+07, 2.400e+07,
       1.700e+07, 3.250e+07, 3.200e+07, 3.400e+07, 3.150e+07, 2.800e+07,
       3.500e+07, 4.150e+07, 3.850e+07, 1.400e+07, 1.500e+07, 1.450e+07,
       2.050e+07, 2.250e+07, 7.000e+06, 2.100e+07, 

In [991]:
#df.loc[df['value_eur'].isna(), 'value_eur'] = 0.0

In [992]:
df[df['release_clause_eur'].isna()]

,player_id,fifa_version,fifa_update,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,...,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,goals,assists
60,232488.0,23,1.0,C. Romero,Cristian Gabriel Romero,CB,83.0,88.0,49500000.0,55000.0,...,73.0,46.0,57.0,65.0,85.0,82.0,42.0,48.0,0,1
98,247394.0,23,1.0,D. Kulusevski,Dejan Kulusevski,"RW, CF",81.0,87.0,41500000.0,105000.0,...,78.0,77.0,79.0,83.0,56.0,76.0,83.0,77.0,2,7
142,202048.0,23,1.0,C. Coady,Conor Coady,CB,79.0,79.0,14500000.0,80000.0,...,47.0,57.0,75.0,66.0,79.0,73.0,67.0,51.0,1,1
164,233306.0,23,1.0,D. Henderson,Dean Henderson,GK,79.0,82.0,18000000.0,76000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,9.0,0,0
184,220440.0,23,1.0,C. Lenglet,Clément Nicolas Laurent Lenglet,CB,78.0,81.0,16000000.0,105000.0,...,68.0,45.0,69.0,64.0,77.0,76.0,55.0,39.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5943,223984.0,15,1.0,R. Fallowfield,Ryan Jack Glenn Fallowfield,RM,52.0,65.0,40000.0,2000.0,...,66.0,44.0,45.0,58.0,33.0,49.0,45.0,41.0,0,0
5944,210424.0,15,1.0,K. Kennedy,Kieran Kennedy,CB,51.0,62.0,40000.0,2000.0,...,53.0,25.0,31.0,38.0,52.0,58.0,28.0,21.0,0,0
5945,207602.0,15,1.0,J. Gordon,Jaanai Gordon,ST,50.0,65.0,30000.0,2000.0,...,80.0,49.0,42.0,57.0,26.0,43.0,45.0,54.0,0,0
5946,220015.0,15,1.0,B. Lewis,Bradley Lewis,CB,49.0,65.0,20000.0,2000.0,...,56.0,28.0,28.0,30.0,49.0,56.0,23.0,25.0,0,0


In [993]:
df[df['value_eur'].isna()]

,player_id,fifa_version,fifa_update,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,...,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,goals,assists
4225,109852.0,17,1.0,G. McAuley,Gareth McAuley,CB,76.0,76.0,NaN,30000.0,...,42.0,43.0,50.0,42.0,78.0,76.0,49.0,50.0,6,1
4354,148569.0,17,1.0,M. Wasilewski,Marcin Ryszard Wasilewski,CB,72.0,72.0,NaN,30000.0,...,39.0,55.0,56.0,40.0,71.0,73.0,65.0,47.0,0,0
4788,26992.0,16,1.0,S. Distin,Sylvain Distin,CB,77.0,77.0,NaN,30000.0,...,68.0,36.0,56.0,54.0,79.0,78.0,40.0,30.0,0,0
5425,26992.0,15,1.0,S. Distin,Sylvain Distin,CB,77.0,77.0,NaN,30000.0,...,71.0,35.0,55.0,56.0,78.0,79.0,39.0,29.0,0,0
5747,140975.0,15,1.0,M. Duff,Michael Duff,CB,67.0,67.0,NaN,5000.0,...,43.0,33.0,50.0,49.0,67.0,62.0,48.0,26.0,0,0


In [994]:
df[df['wage_eur'].isna()]

,player_id,fifa_version,fifa_update,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,...,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,goals,assists


In [995]:
df['wage_eur'].unique()

array([350000., 220000., 230000., 270000., 240000., 210000., 190000.,
       250000., 260000.,  80000., 150000., 175000., 185000., 200000.,
       115000., 180000., 195000., 130000., 170000., 165000., 110000.,
       120000., 140000., 155000.,  95000., 105000.,  83000., 135000.,
        55000.,  43000., 100000.,  85000.,  45000.,  62000.,  76000.,
        70000.,  64000.,  47000.,  75000.,  69000.,  74000.,  82000.,
        66000.,  88000.,  90000.,  50000.,  46000., 125000.,  52000.,
        73000.,  84000.,  77000.,  48000.,  65000.,  58000.,  29000.,
        79000.,  71000.,  49000.,  81000.,  72000.,  86000.,  89000.,
        60000.,  51000.,  37000.,  63000.,  57000.,  78000.,  59000.,
        44000.,  36000.,  56000.,  40000.,  26000.,  31000.,  53000.,
        38000.,  35000.,  68000.,  25000.,  41000.,  34000.,  42000.,
        33000.,    900.,  17000.,  61000.,  28000.,  22000.,  23000.,
        39000.,  54000.,  24000.,   9000.,  30000.,  32000.,  27000.,
        12000.,  130

In [996]:
df['weight_kg'].unique()

array([ 70.,  83.,  92.,  71.,  78.,  89.,  86.,  91.,  74.,  64.,  94.,
        82.,  76.,  69.,  79.,  67.,  85.,  80.,  68.,  73.,  90.,  81.,
        88.,  77.,  72.,  87.,  65.,  84.,  66.,  61., 100.,  95.,  75.,
        62.,  60.,  59.,  97.,  58.,  93., 101.,  96.,  63.,  99.,  98.,
        56., 110.,  55.,  57.])

In [997]:
cols_for_nn = ['fifa_version', 'player_id'
       'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age', 'height_cm', 'weight_kg', 'club_team_id', 'club_position',
       'club_jersey_number', 
       'nation_jersey_number', 'preferred_foot',
       'weak_foot', 'skill_moves', 'international_reputation', 'work_rate',
       'body_type', 'release_clause_eur', 'pace', 'shooting', 'passing',
       'dribbling', 'defending', 'physic', 'attacking_crossing',
       'attacking_finishing', 'goals', 'assists']

In [998]:
df['club_position'].unique()

array(['RCM', 'ST', 'LCB', 'RW', 'LW', 'GK', 'LB', 'LCM', 'RCB', 'CDM',
       'RB', 'CB', 'SUB', 'RDM', 'CAM', 'RES', 'LDM', 'RWB', 'LM', 'LWB',
       'RM', 'LS', 'RS', 'CF', 'CM'], dtype=object)

In [999]:
unique_first_pos = []
for elt in pos_array:
    elt

In [1000]:
df['international_reputation'].unique()

array([4., 5., 3., 2., 1.])

In [1001]:
df['body_type'].unique()

array(['Unique', 'Lean (170-185)', 'Normal (185+)', 'Normal (170-185)',
       'Lean (185+)', 'Stocky (185+)', 'Stocky (170-185)',
       'Normal (170-)', 'Lean (170-)', 'Stocky (170-)'], dtype=object)

In [1002]:
position_map = {
    'RCM': 0, 'ST': 1, 'LCB': 2, 'RW': 3, 'LW': 4, 'GK': 5, 'LB': 6, 'LCM': 7,
    'RCB': 8, 'CDM': 9, 'RB': 10, 'CB': 11, 'SUB': 12, 'RDM': 13, 'CAM': 14,
    'RES': 15, 'LDM': 16, 'RWB': 17, 'LM': 18, 'LWB': 19, 'RM': 20, 'LS': 21,
    'RS': 22, 'CF': 23, 'CM': 24
}

df['club_position_mapped'] = df['club_position'].map(position_map)

# body_type_map = {
#     'Unique': 0, 'Lean (170-185)': 1, 'Normal (185+)': 2, 'Normal (170-185)': 3,
#     'Lean (185+)': 4, 'Stocky (185+)': 5, 'Stocky (170-185)': 6,
#     'Normal (170-)': 7, 'Lean (170-)': 8, 'Stocky (170-)': 9
# }

# df['body_type'] = df['body_type'].map(body_type_map)


# work_rate_map = {
#     'High/High': 0,
#     'High/Low': 1,
#     'Medium/High': 2,
#     'High/Medium': 3,
#     'Medium/Medium': 4,
#     'Low/High': 5,
#     'Medium/Low': 6,
#     'Low/Low': 7,
#     'Low/Medium': 8
# }

# df['work_rate_mapped'] = df['work_rate'].map(work_rate_map)


In [1003]:
len(df)

5898

In [1004]:
df['primary_position'] = df['player_positions'].apply(lambda x: x.split(',')[0].strip())
df['has_multiple_positions'] = df['player_positions'].apply(lambda x: ',' in x).astype(int)
df['has_multiple_positions']

0       1
1       0
2       0
3       0
4       1
       ..
5943    0
5944    0
5945    0
5946    0
5947    1
Name: has_multiple_positions, Length: 5898, dtype: int32

In [1005]:
df['primary_position_mapped'] = df['primary_position'].map(position_map)

In [1006]:
df['primary_position_mapped'].unique()

array([24,  1, 11,  3,  4,  5,  9,  6, 14, 10, 23, 18, 17, 19, 20],
      dtype=int64)

In [1007]:
df['club_team_id'].unique()

array([1.000e+01, 1.100e+01, 9.000e+00, 1.800e+01, 5.000e+00, 9.500e+01,
       1.300e+01, 2.000e+00, 1.000e+00, 1.900e+01, 1.100e+02, 1.799e+03,
       7.000e+00, 1.943e+03, 1.700e+01, 1.440e+02, 8.000e+00, 1.808e+03,
       1.400e+01, 1.925e+03, 1.796e+03, 1.795e+03, 1.792e+03, 1.794e+03,
       1.090e+02, 1.939e+03, 1.961e+03, 1.806e+03, 1.960e+03, 1.200e+01,
       1.060e+02, 1.952e+03, 1.500e+01])

In [1008]:
df['club_contract_valid_until_year'].unique()

array([2025., 2023., 2024., 2026., 2027., 2028., 2022., 2021., 2020.,
       2019., 2018., 2017., 2016., 2015., 2014.])

In [1009]:
df['years_remaining'] = df['club_contract_valid_until_year'] - 2000 - df['fifa_version']

In [1010]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# ---- Step 1: Handle NaNs ----
# Optionally fill with low-end values (e.g., min or 1st percentile)
for col in ['value_eur', 'wage_eur']:
    if df[col].isna().any():
        low_val = df[col].quantile(0.01)  # better than just min()
        df[col] = df[col].fillna(low_val)

# ---- Step 2: Log Transform ----
df['log_value_eur'] = np.log1p(df['value_eur'])
df['log_wage_eur'] = np.log1p(df['wage_eur'])

# ---- Step 3: Normalize ----
scaler = StandardScaler()
df[['log_value_eur', 'log_wage_eur']] = scaler.fit_transform(
    df[['log_value_eur', 'log_wage_eur']]
)
df[['age', 'height_cm', 'weight_kg']] = scaler.fit_transform(df[['age', 'height_cm', 'weight_kg']])


In [1011]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = encoder.fit_transform(df[['work_rate']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['work_rate']))
encoded_df

,work_rate_High/High,work_rate_High/Low,work_rate_High/Medium,work_rate_Low/High,work_rate_Low/Low,work_rate_Low/Medium,work_rate_Medium/High,work_rate_Medium/Low,work_rate_Medium/Medium
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
5893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5895,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5896,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [1012]:
# Combine with original DataFrame
df = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)
df.drop(columns=['work_rate'], inplace=True)

In [1013]:
encoded = encoder.fit_transform(df[['body_type']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['body_type']))
df = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)
df.drop(columns=['body_type'], inplace=True)

In [1014]:
# Ensure integer mapping of IDs (needed for Embedding input)
df['player_id'] = df['player_id'].astype('category').cat.codes
df['club_team_id'] = df['club_team_id'].astype('category').cat.codes

In [1015]:
df['log_wage_eur'] 

0       2.054271
1       1.665869
2       1.703054
3       1.837184
4       1.738656
          ...   
5893   -2.265784
5894   -2.265784
5895   -2.265784
5896   -2.265784
5897   -2.265784
Name: log_wage_eur, Length: 5898, dtype: float64

In [1016]:
df['log_value_eur']

0       2.118835
1       1.521975
2       2.061545
3       2.163281
4       2.080216
          ...   
5893   -2.770592
5894   -2.770592
5895   -2.948720
5896   -3.199774
5897   -2.948720
Name: log_value_eur, Length: 5898, dtype: float64

In [1017]:
df.columns

Index(['player_id', 'fifa_version', 'fifa_update', 'short_name', 'long_name',
       'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age', 'dob', 'height_cm', 'weight_kg', 'league_id', 'league_name',
       'league_level', 'club_team_id', 'club_name', 'club_position',
       'club_jersey_number', 'club_contract_valid_until_year',
       'nationality_name', 'nation_jersey_number', 'preferred_foot',
       'weak_foot', 'skill_moves', 'international_reputation',
       'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
       'goals', 'assists', 'club_position_mapped', 'primary_position',
       'has_multiple_positions', 'primary_position_mapped', 'years_remaining',
       'log_value_eur', 'log_wage_eur', 'work_rate_High/High',
       'work_rate_High/Low', 'work_rate_High/Medium', 'work_rate_Low/High',
       'work_rate_Low/Low', 'work_rate_Low/Medium', 'work_rate_Medium/

In [1018]:
df[(df['fifa_version']==22) & (df['goals'] == 2) & (df['assists'] == 1)][['player_id', 'short_name', 'long_name', 'goals', 'assists']]

,player_id,short_name,long_name,goals,assists
665,393,E. Cavani,Edinson Roberto Cavani Gómez,2,1
686,96,Fernandinho,Fernando Luiz Rosa,2,1
697,1063,T. Partey,Thomas Partey,2,1
722,1922,Ferran Torres,Ferran Torres García,2,1
740,198,João Moutinho,João Filipe Iria Santos Moutinho,2,1
772,752,A. Oxlade-Chamberlain,Alex Oxlade-Chamberlain,2,1
800,431,M. Sissoko,Moussa Sissoko,2,1
840,849,S. Armstrong,Stuart Armstrong,2,1
857,1427,M. Holgate,Mason Holgate,2,1
930,566,J. Shelvey,Jonjo Shelvey,2,1


In [1019]:
use_cols = ['player_id', 'fifa_version',
       'overall', 'potential',
       'age', 'height_cm', 'weight_kg', 'club_team_id',
       'club_jersey_number',
       'preferred_foot',
       'weak_foot', 'skill_moves', 'international_reputation',
       'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
       'goals', 'assists', 'club_position_mapped', 'primary_position',
       'has_multiple_positions', 'primary_position_mapped', 'years_remaining',
       'log_value_eur', 'log_wage_eur', 'work_rate_High/High',
       'work_rate_High/Low', 'work_rate_High/Medium', 'work_rate_Low/High',
       'work_rate_Low/Low', 'work_rate_Low/Medium', 'work_rate_Medium/High',
       'work_rate_Medium/Low', 'work_rate_Medium/Medium',
       'body_type_Lean (170-)', 'body_type_Lean (170-185)',
       'body_type_Lean (185+)', 'body_type_Normal (170-)',
       'body_type_Normal (170-185)', 'body_type_Normal (185+)',
       'body_type_Stocky (170-)', 'body_type_Stocky (170-185)',
       'body_type_Stocky (185+)', 'body_type_Unique']

In [1020]:
df = df[use_cols]

In [1021]:
df.to_csv("cleaned_prem_data.csv", index=False)

In [1022]:
conn.close()